# Traffic Incident Management in Nashville, TN

### Loading the libraries

In [3]:
#from shapely.geometry import Point
import pandas as pd
#import geopandas as gpd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
#import folium
import datetime
import math
import re
#from folium.plugins import MarkerCluster
#from folium.plugins import FastMarkerCluster

%matplotlib inline

### Importing the data

In [4]:
traffic_df = pd.read_excel('../data/MASTER File.xlsx')
d_weather = pd.read_excel('../data/Daily Weather Formatted.xlsx')
h_weather = pd.read_excel('../data/Hourly Weather Formatted.xlsx')

### Reviewing the Traffic Data

In [5]:
print(traffic_df.shape)

(33730, 21)


In [6]:
#keep in case
#print(traffic_df.crs)

AttributeError: 'DataFrame' object has no attribute 'crs'

In [17]:
traffic_df.head()

,Region,Incident ID,Incident Type,Full Location,Start Date,Start Time,Duration (mins),Lanes\nBlocked,Blockage\nDuration,Resp Time,...,Key,County,Interstate,Route,Direction,Unnamed: 16,Description,Inc Mile Marker,Latitude,Longitude
0,3,25986,Overturned Vehicle,"ROBERTSON on Interstate 65 Southbound, At EXIT...",44362.657535,44362.657535,338.200000,2,320.266667,0.116667,...,21476,ROBERTSON,I-65S,I-65,Southbound,At,EXIT 104 SR-257 / BETHEL RD. / RIDGETOP,104.0,36.419130,-86.717748
1,3,769,MultiVehicle Crash,"DAVIDSON on Interstate 65 Northbound, At EXIT ...",44147.647211,44147.647211,19.400000,1,16.333333,0.333333,...,32,DAVIDSON,I-65N,I-65,Northbound,At,EXIT 4 RIVERGATE PKWY. / GOODLETTSVILLE,4.0,35.048795,-86.880272
2,3,5254,MultiVehicle Crash,"DAVIDSON on Interstate 65 Northbound, At EXIT ...",44188.653657,44188.653657,32.466667,2,15.116667,1.116667,...,3705,DAVIDSON,I-65N,I-65,Northbound,At,EXIT 4 RIVERGATE PKWY. / GOODLETTSVILLE,4.0,35.048795,-86.880272
3,3,9621,MultiVehicle Crash,"DAVIDSON on Interstate 65 Northbound, At EXIT ...",44232.863935,44232.863935,33.066667,0,NaN,NaN,...,7579,DAVIDSON,I-65N,I-65,Northbound,At,EXIT 4 RIVERGATE PKWY. / GOODLETTSVILLE,4.0,35.048795,-86.880272
4,3,10090,Disabled Vehicle,"DAVIDSON on Interstate 65 Northbound, At EXIT ...",44237.649907,44237.649907,197.133333,0,NaN,3.016667,...,7957,DAVIDSON,I-65N,I-65,Northbound,At,EXIT 4 RIVERGATE PKWY. / GOODLETTSVILLE,4.0,35.048795,-86.880272


In [16]:
traffic_df.tail()

,Region,Incident ID,Incident Type,Full Location,Start Date,Start Time,Duration (mins),Lanes\nBlocked,Blockage\nDuration,Resp Time,...,Key,County,Interstate,Route,Direction,Unnamed: 16,Description,Inc Mile Marker,Latitude,Longitude
33725,3,106595,MultiVehicle Crash,"DAVIDSON on Interstate 40 Westbound, Before MI...",44500.680486,44500.680486,38.016667,0,NaN,8.900000,...,37419,DAVIDSON,I-40W,I-40,Westbound,Before,MILE MARKER 217.4,217.4,36.142999,-86.650442
33726,3,106603,Debris on Roadway,"DAVIDSON on Interstate 40 Westbound, Before MI...",44500.730648,44500.730648,1.383333,0,NaN,0.166667,...,37427,DAVIDSON,I-40W,I-40,Westbound,Before,MILE MARKER 213,213.0,36.140720,-86.727637
33727,3,106609,Debris on Roadway,"DAVIDSON on Interstate 40 Eastbound, Before MI...",44500.754977,44500.754977,3.283333,1,1.866667,2.550000,...,37433,DAVIDSON,I-40E,I-40,Eastbound,Before,MILE MARKER 205.4,205.4,36.154497,-86.832437
33728,3,106610,Disabled Vehicle,"DAVIDSON on Interstate 40 Eastbound, Beyond MI...",44500.759387,44500.759387,50.466667,0,NaN,0.033333,...,37434,DAVIDSON,I-40E,I-40,Eastbound,Beyond,MILE MARKER 218.8,218.8,36.154202,-86.631745
33729,3,106623,MultiVehicle Crash,"DAVIDSON on Interstate 40 Eastbound, Before MI...",44500.899479,44500.899479,48.816667,2,48.366667,0.016667,...,37443,DAVIDSON,I-40E,I-40,Eastbound,Before,MILE MARKER 214.4,214.4,36.143086,-86.705514


In [8]:
traffic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33730 entries, 0 to 33729
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Region             33730 non-null  int64  
 1   Incident ID        33730 non-null  int64  
 2   Incident Type      33730 non-null  object 
 3   Full Location      33730 non-null  object 
 4   Start Date         33730 non-null  float64
 5   Start Time         33730 non-null  float64
 6   Duration (mins)    33730 non-null  float64
 7   Lanes
Blocked      33730 non-null  int64  
 8   Blockage
Duration  9208 non-null   float64
 9   Resp Time          29366 non-null  float64
 10  Rural/
Urban       33730 non-null  object 
 11  Key                33730 non-null  int64  
 12  County             33730 non-null  object 
 13  Interstate         33730 non-null  object 
 14  Route              33730 non-null  object 
 15  Direction          33730 non-null  object 
 16  Unnamed: 16        337

In [18]:
traffic_df.columns

Index(['Region', 'Incident ID', 'Incident Type', 'Full Location ',
       'Start Date', 'Start Time', 'Duration (mins)', 'Lanes\nBlocked',
       'Blockage\nDuration', 'Resp Time', 'Rural/\nUrban', 'Key ', 'County',
       'Interstate', 'Route', 'Direction', 'Unnamed: 16', 'Description',
       'Inc Mile Marker', 'Latitude', 'Longitude'],
      dtype='object')

In [ ]:
traffic_df['column name'].value_counts()

In [ ]:
traffic_df['column name'].isnull().value_counts()

In [ ]:
traffic_df['column'].unique()

In [12]:
davidson = traffic_df[traffic_df['County']=='Davidson']

,Region,Incident ID,Incident Type,Full Location,Start Date,Start Time,Duration (mins),Lanes\nBlocked,Blockage\nDuration,Resp Time,...,Key,County,Interstate,Route,Direction,Unnamed: 16,Description,Inc Mile Marker,Latitude,Longitude


### geospatial coding. update 'geometry' column

In [ ]:
traffic_df['geometry'] = traffic_df.apply(lambda x: Point((float(x.Longitude),
                                                   float(x.Latitude))),
                                  axis=1)

In [ ]:
traffic_df.head()

In [ ]:
traffic_geo = gpd.GeoDataFrame(traffic_df)

In [ ]:
traffic_geo.head()

In [ ]:
type(traffic_geo)

### Save the number of rows and columns as a tuple

In [ ]:
rows_and_cols = traffic_df.shape
print('There are {} rows and {} columns.\n'.format(
    rows_and_cols[0], rows_and_cols[1]))

In [ ]:
# compile the list of dataframes to be merged
data_frames = [car_accident0, car_accident1, car_accident2,car_accident3, car_accident4]

In [ ]:
traffic_2021 = pd.concat([traffic_df, h_weather, d_weather], sort=True)
traffic_2021.head(2)

In [ ]:
#The amount in brackets will populate. 
print('There are {} accidents in the data'.format(len(car_accidents_2010_2020)))

In [ ]:
#dropping columns
pd.set_option('display.max_columns', 1000)
df.drop(['property_damage', 'accident_number', 'reporting_officer', 'collision_type', 'weather', 'illumination', 
         'harmful_codes', 'state_name', 'rpa', 'geocoded_column', ':@computed_region_wvby_4s8j', 
         ':@computed_region_3aw5_2wv7', ':@computed_region_p6sk_2acq', ':@computed_region_gxvr_9jxz',
         'property_damage'], axis=1, inplace=True)
df.tail()

# columns listed
'Region', 'Incident ID', 'Incident Type', 'Full Location ',
       'Start Date', 'Start Time', 'Duration (mins)', 'Lanes\nBlocked',
       'Blockage\nDuration', 'Resp Time', 'Rural/\nUrban', 'Key ', 'County',
       'Interstate', 'Route', 'Direction', 'Unnamed: 16', 'Description',
       'Inc Mile Marker', 'Latitude', 'Longitude'

### top _ sorted 

In [ ]:
gdp_and_internet_use_2014.sort_values(by = 'Internet_Users_Pct', ascending = False).head()

### Reviewing Weather Data

In [ ]:
h_weather.head()

In [ ]:
h_weather.columns

In [ ]:
d_weather.head()

In [ ]:
d_weather.columns

### Date/Time

In [ ]:
car_accidents_2010_2020['DateTime'] = pd.to_datetime(car_accidents_2010_2020['Date and Time'])
car_accidents_2010_2020['date'] = car_accidents_2010_2020['DateTime'].dt.date
car_accidents_2010_2020['time'] = car_accidents_2010_2020['DateTime'].dt.time

In [ ]:
car_accidents_2010_2020 = car_accidents_2010_2020.set_index('DateTime')
car_accidents_2010_2020.head(3)

### Histographs

In [ ]:
traffic_df['column name'].value_counts()

In [ ]:
traffic_df['number_of_motor_vehicles'] = pd.to_numeric(traffic_df['number_of_motor_vehicles'])  
                                                                        
traffic_df.hist(['number_of_motor_vehicles'])

In [ ]:
car_accidents_2010_2020['Collision Type'].value_counts(normalize=True).sort_index().plot.bar()
plt.grid()
plt.title('collision Type')
plt.xlabel('Collision Type')
plt.ylabel('Fraction');

In [ ]:
car_accidents_2010_2020['Collision Type'].value_counts().plot.bar()
plt.grid()
plt.title('collision Type')
plt.xlabel('Collision Type')
plt.ylabel('Fraction');

In [ ]:
car_accidents_2010_2020['Collision Type Description'].value_counts(normalize=True).sort_index().plot.bar()
plt.grid()
plt.title('Collision Type Description')
plt.xlabel('Collision Type Description')
plt.ylabel('Fraction');

In [ ]:
car_accidents_2010_2020['Collision Type Description'].value_counts().plot.bar()
plt.grid()
plt.title('Collision Type Description')
plt.xlabel('Collision Type Description')
plt.ylabel('Total count');

In [ ]:
car_accidents_2010_2020['Weekday'] = car_accidents_2010_2020.index.weekday_name
weekday = car_accidents_2010_2020.groupby('Weekday')['Accident Number'].count()
weekday = weekday/weekday.sum()
dayOfWeek=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
weekday[dayOfWeek].plot.bar()
plt.title('Acccidents by Weekday')
plt.xlabel('Weekday')
plt.ylabel('Accidents');

In [ ]:
years = ['2010','2011','2012','2013', '2014', '2015', '2016', '2017', '2018', '2019']
fig, _ = plt.subplots(1,10,figsize=(21,7), sharex='all', sharey='all')

fig.suptitle('Acccidents by Weekday for Different Years')
plt.xlabel('Weekday')
plt.ylabel('Accidents')
for i, year in enumerate(years,1):
    weekday = car_accidents_2010_2020.loc[year].groupby('Weekday')['Accident Number'].count()
    weekday = weekday/weekday.sum()
    dayOfWeek=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    plt.subplot(1,10,i)
    plt.title(year)
    weekday[dayOfWeek].plot.bar()

### Create scatterplots

In [ ]:
plt.figure(figsize = (15,10))
sns.scatterplot(x = 'Internet_Users_Pct', y = 'GDP_Per_Capita', data = gdp_and_internet_use_2004, alpha = 0.9)
plt.title('2004 Percent Using the Internet vs GDP Per Capita', fontsize = 15)
plt.xlabel('Percentage of Internet Users', fontsize = 12)
plt.ylabel('GDP Per Capita', fontsize = 12)

In [ ]:
# distribution of gdp per capita 
ax = sns.distplot(gdp_and_internet_use.gdp_percapita,
                 bins = 25,
                  hist_kw={'alpha' :.2})
ax.set(xlabel = 'GDP per Capita', ylabel = 'Frequency')

In [ ]:
#internet use
ax = sns.distplot(gdp_and_internet_use.Internet_Users_Pct,
                 bins = 25,
                  color  = 'green',
                  hist_kw={'alpha':.4})
ax.set(xlabel = 'Internet User Pct', ylabel = 'Frequency')

In [ ]:
years = ['2010','2011','2012','2013', '2014', '2015', '2016', '2017', '2018', '2019']
fig, _ = plt.subplots(1,10, figsize=(44,8), sharex='all', sharey='all')

fig.suptitle('Acccidents by month for Different Years')
plt.xlabel('month')
plt.ylabel('Accidents')
for i, year in enumerate(years,1):
    plt.subplot(1,10,i)
    sample = car_accidents_2010_2020.loc[year]['Accident Number'].resample('M').count()
    sample.plot()
    plt.ylim(0,8000)
    plt.title('Accidents, {} count'.format(text))
    plt.xlabel('Year')
    plt.ylabel('Accident Count');